In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
heart_df = pd.read_csv("../datasets/heart.csv")

In [5]:
heart_df.sample(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
239,35,1,0,126,282,0,0,156,1,0.0,2,0,3,0
21,44,1,2,130,233,0,1,179,1,0.4,2,0,2,1
256,58,1,0,128,259,0,0,130,1,3.0,1,2,3,0
50,51,0,2,130,256,0,0,149,0,0.5,2,0,2,1
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3,0
70,54,1,2,120,258,0,0,147,0,0.4,1,0,3,1
78,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
123,54,0,2,108,267,0,0,167,0,0.0,2,0,2,1
108,50,0,1,120,244,0,1,162,0,1.1,2,0,2,1
160,56,1,1,120,240,0,1,169,0,0.0,0,0,2,1


In [6]:
heart_df.target.value_counts()

target
1    165
0    138
Name: count, dtype: int64

In [7]:
heart_df.isnull()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,False,False,False,False,False,False,False,False,False,False,False,False,False,False
299,False,False,False,False,False,False,False,False,False,False,False,False,False,False
300,False,False,False,False,False,False,False,False,False,False,False,False,False,False
301,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [8]:
heart_df.shape

(303, 14)

In [9]:
heart_df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
heart_df.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trestbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalach,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exang,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


In [11]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [12]:
X = heart_df.drop(columns=["target"],axis=1)
y = heart_df['target']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)

Using Logistic Regression With fine tuning and getting performance

In [14]:
param_grid = {"C":[0.01,0.1,1,10,100]}
lr = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5, scoring="accuracy", n_jobs= -1,)
lr.fit(X_train,y_train)
prediction = lr.predict(X_test)

#Print results and scores
print(f"Best params : {lr.best_params_} , best score: {lr.best_score_}")
print(f"Accurecy test :{accuracy_score(y_test,prediction)}")
print(f"recall score :{recall_score(y_test,prediction)}")
print(f"Precision score :{precision_score(y_test,prediction)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test,prediction)}")

#print whole classifiaction report
print(f"This is the full classifiaction report:\n{classification_report(y_test,prediction)}")

Best params : {'C': 100} , best score: 0.814455782312925
Accurecy test :0.8688524590163934
recall score :0.8611111111111112
Precision score :0.9117647058823529
Confusion Matrix:
 [[22  3]
 [ 5 31]]
This is the full classifiaction report:
              precision    recall  f1-score   support

           0       0.81      0.88      0.85        25
           1       0.91      0.86      0.89        36

    accuracy                           0.87        61
   macro avg       0.86      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61



In [15]:
param_grid = {
    "max_depth": [3, 5, 10, None], 
    "min_samples_split": [2, 5, 10], 
    "criterion": ["gini", "entropy"]
}

dt = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
dt.fit(X_train, y_train)
prediction = dt.predict(X_test)

# Print results
print(f"Best params: {dt.best_params_} and best score: {dt.best_score_}")
print(f"Test Accuracy: {accuracy_score(y_test, prediction):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction))
print(f"Recall Score: {recall_score(y_test, prediction, average='weighted'):.4f}")
print(f"Precision Score: {precision_score(y_test, prediction, average='weighted'):.4f}")
print("Classification Report:\n", classification_report(y_test, prediction))

Best params: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2} and best score: 0.7850340136054421
Test Accuracy: 0.8525
Confusion Matrix:
 [[19  6]
 [ 3 33]]
Recall Score: 0.8525
Precision Score: 0.8533
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.76      0.81        25
           1       0.85      0.92      0.88        36

    accuracy                           0.85        61
   macro avg       0.85      0.84      0.84        61
weighted avg       0.85      0.85      0.85        61



In [17]:
param_grid = {
    "n_estimators": [50, 100, 200], 
    "max_depth": [5, 10, 20], 
    "min_samples_split": [2, 5]
}

rf = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
rf.fit(X_train, y_train)
prediction = rf.predict(X_test)

# Print results
print(f"Best params: {rf.best_params_} and best score: {rf.best_score_}")
print(f"Test Accuracy: {accuracy_score(y_test, prediction):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction))
print(f"Recall Score: {recall_score(y_test, prediction, average='weighted'):.4f}")
print(f"Precision Score: {precision_score(y_test, prediction, average='weighted'):.4f}")
print("Classification Report:\n", classification_report(y_test, prediction))

Best params: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 100} and best score: 0.8308673469387754
Test Accuracy: 0.9016
Confusion Matrix:
 [[24  1]
 [ 5 31]]
Recall Score: 0.9016
Precision Score: 0.9109
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89        25
           1       0.97      0.86      0.91        36

    accuracy                           0.90        61
   macro avg       0.90      0.91      0.90        61
weighted avg       0.91      0.90      0.90        61



In [18]:
model = svm.SVC(kernel='linear', class_weight='balanced')

model.fit(X_train,y_train)
prediction = model.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, prediction):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction))
print(f"Recall Score: {recall_score(y_test, prediction, average='weighted'):.4f}")
print(f"Precision Score: {precision_score(y_test, prediction, average='weighted'):.4f}")
print("Classification Report:\n", classification_report(y_test, prediction))

Test Accuracy: 0.9016
Confusion Matrix:
 [[23  2]
 [ 4 32]]
Recall Score: 0.9016
Precision Score: 0.9046
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.92      0.88        25
           1       0.94      0.89      0.91        36

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61



In [19]:
param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11],  
    "weights": ["uniform", "distance"],  
    "metric": ["euclidean", "manhattan", "minkowski"]  
}


knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
knn.fit(X_train, y_train)  # Train KNN on scaled data

# Predict on test data
prediction = knn.predict(X_test)

# Print results
print(f"Best params: {knn.best_params_} and best score: {knn.best_score_}")
print(f"Test Accuracy: {accuracy_score(y_test, prediction):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, prediction))
print(f"Recall Score: {recall_score(y_test, prediction, average='weighted'):.4f}")
print(f"Precision Score: {precision_score(y_test, prediction, average='weighted'):.4f}")
print("Classification Report:\n", classification_report(y_test, prediction))

Best params: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'} and best score: 0.685969387755102
Test Accuracy: 0.7049
Confusion Matrix:
 [[15 10]
 [ 8 28]]
Recall Score: 0.7049
Precision Score: 0.7021
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.60      0.62        25
           1       0.74      0.78      0.76        36

    accuracy                           0.70        61
   macro avg       0.69      0.69      0.69        61
weighted avg       0.70      0.70      0.70        61



In [103]:
pip install pickel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickel (from versions: none)
ERROR: No matching distribution found for pickel


In [104]:
import pickle
filename = "heart_disease_model"
pickle.dump(rf.best_estimator_,open(filename,"wb"))
